In [4]:
import sys
sys.path.append('/home/ruyogagp/medical_interpretability')
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from cga import cga
import pandas as pd
import neptune.new as neptune
import wandb
import pytorch_lightning as pl
import torch.nn as nn
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from source.tasks import DeepSurv

from captum.attr import *
from captum.metrics import *
from captum._utils.models.linear_model import SkLearnLinearRegression
from source.wrappers import ForwardWrapper
import os
import seaborn as sns

In [5]:
RESULTS_DIR = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/results/interpretability/resample_multiplicities'
DATA_DIR = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/resample_multiplicities'

## Load from wandb

In [6]:
# TODO: this works if the second tag is the experiment identifier
def load_details(run):
    dir = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/resample_multiplicities'
    path = f'{dir}/{run.tags[0]}_attribute_details.csv'
    details_df = pd.read_csv(path)
    return details_df, path

def output_diff(model, orig_features, resampled_features):
    orig_output, *_ = model(orig_features)
    resampled_output, *_ = model(resampled_features)
    with torch.no_grad():
        diff = torch.sub(orig_output, resampled_output)
    return diff.detach().numpy()

def load_attributions(run, method):
    attr_x = np.genfromtxt(run.config[f'{method}_x_path'], delimiter=',')
    attr_y = np.genfromtxt(run.config[f'{method}_y_path'], delimiter=',')
    return attr_x, attr_y

def calculate_error(details_x, details_y):
    for detail in [details_x, details_y]:
        outlier_cs = np.percentile(detail.change_slope, [0.5, 99.5])
        clipped_cs = np.clip(detail.change_slope.to_numpy(), *outlier_cs)
        cs_norm = clipped_cs / abs(clipped_cs).max()
        detail.loc[:,'change_slope_norm'] = cs_norm

        outlier_attr = np.percentile(detail.attribution, [0.5, 99.5])
        clipped_attr = np.clip(detail.attribution.to_numpy(), *outlier_attr)
        detail.loc[:,'attribution_norm'] = clipped_attr / abs(clipped_attr).max()
        detail.loc[:, 'error'] = abs(detail.attribution_norm - detail.change_slope_norm)

def view_error(project, id, method):
    api = wandb.Api()
    run = api.run(f"cardiors/{project}/{id}")
    x, y = change_slope_singular(run, method)
    y.plot.scatter('x_orig', 'y_orig', c='error', cmap='PuBu')
    x.plot.scatter('x_orig', 'y_orig', c='error', cmap='PuBu')

In [7]:
def change_slope_singular(run, method):
    """
    :param run: wandb run object
    :param method: attribution method to calculate change_slope for
    :return: None -> loads the details dataframe of the experiment and writes the change slope dataframe to {experiment}/{change_slope}/x_{method}_{seed}.csv
    """
    details = load_details(run)
    model = DeepSurv.load_from_checkpoint(run.config['checkpoint_path'])
    details_x = details[details['modified_attribute'] == 'noise_x']
    details_y = details[details['modified_attribute'] == 'noise_y']
    attr_x, attr_y = load_attributions(run, method)
    for detail in [details_x, details_y]:
        # get input for model difference
        original_values = torch.Tensor(detail[['x_orig', 'y_orig']].to_numpy(dtype='float32'))
        # TODO: loop and change per intervention id, also store intermediate results in a dictionary
        resampled_values = torch.Tensor(detail[['x_do', 'y_do']].to_numpy(dtype='float32'))
        # get resampling difference based on the modified attribute
        if detail.loc[detail.index.min(), ['modified_attribute']].item() == 'noise_x':
            # TODO: Store resampling difference in dict {resampling_difference0, resampling_difference1, etc}
            detail.loc[:, 'resampling_diff'] = original_values[:, 0] - resampled_values[:, 0]
            #TODO: fix this
            detail.loc[:, 'attribution'] = attr_x[:, 0]
        elif detail.loc[detail.index.min(), ['modified_attribute']].item() == 'noise_y':
            detail.loc[:, 'resampling_diff'] = original_values[:, 1] - resampled_values[:, 1]
            detail.loc[:, 'attribution'] = attr_y[:, 1]
        else:
            raise Exception

        # calculate change slope

        detail.loc[:, 'model_diff']= output_diff(model, original_values, resampled_values)
        detail.loc[:, 'change_slope'] = detail.model_diff / detail.resampling_diff
    calculate_error(details_x, details_y)

    # save dataframes
    seed = eval(run.config['_content']['experiment'])['datamodule_kwargs']['seed']
    filepath_x = '/'.join(run.config[f'{method}_x_path'].split('/')[0:-1] + ['change_slope', f'x_{method}_{seed}.csv'])
    filepath_y = '/'.join(run.config[f'{method}_x_path'].split('/')[0:-1] + ['change_slope', f'y_{method}_{seed}.csv'])
    details_x.to_csv(filepath_x)
    details_y.to_csv(filepath_y)

    return details_x, details_y

In [8]:
def get_mean_error(experiment_id, method):
    """
    :param experiment_id: experiment id
    :param method: attribution method
    :return: None -> prints the error graph
    """
    api = wandb.Api()
    runs =  api.runs('cardiors/interpretability',
                     filters={"$and": [{'tags': f'{experiment_id}'}, {'state': 'finished'}]})
    print('len runs:', len(runs))
    for run in runs:
        deets_x, deets_y = change_slope_singular(run, method)


    valid_attr_x = []
    valid_attr_y = []
    dir = os.listdir(f'/data/analysis/ag-reils/ag-reils-shared/cardioRS/results/interpretability/correlation_case/{experiment_id}/change_slope/')
    change_slope_path = f'/data/analysis/ag-reils/ag-reils-shared/cardioRS/results/interpretability/correlation_case/{experiment_id}/change_slope/'
    for attr in dir:
        dim = attr.split('_')[0]
        name = attr.split('_')[1]
        seed = attr.split('_')[2]
        if dim == 'x' and name == method:
            valid_attr_x.append(pd.read_csv(f'{change_slope_path}/{attr}', index_col=0))
        elif dim == 'y' and name == method:
            valid_attr_y.append(pd.read_csv(f'{change_slope_path}/{attr}', index_col=0))

    err_x = None
    err_y = None
    for attr in valid_attr_x:
        error_x = attr['error'].to_numpy().reshape(-1, 1)
        err_x = error_x if err_x is None else np.concatenate((err_x, error_x), axis=1)
    mean_error_x = np.mean(err_x, axis=1)
    deets_x.loc[:, 'mean_error'] = mean_error_x

    for attr in valid_attr_y:
        error_y = attr['error'].to_numpy().reshape(-1, 1)
        if np.isnan(error_y).any():
            continue
        err_y = error_y if err_y is None else np.concatenate((err_y, error_y), axis=1)
    mean_error_y = np.mean(err_y, axis=1)
    deets_y.loc[:, 'mean_error'] = mean_error_y

    # plot figures
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))
    fig.suptitle(method, fontsize=35)
    dim0_mean = deets_x['mean_error'].mean()
    dim1_mean = deets_y['mean_error'].mean()
    ax1.set_title(f"dimension 0 error: {dim0_mean}", fontdict=dict(fontsize=20))
    ax2.set_title(f"dimension 1 error: {dim1_mean}", fontdict=dict(fontsize=20))
    deets_x.plot.scatter('x_orig', 'y_orig', c='mean_error', cmap='PuBu', vmax=1.9, vmin=0.0, ax=ax1)
    deets_y.plot.scatter('x_orig', 'y_orig', c='mean_error', cmap='PuBu', vmax=1.9, vmin=0.0, ax=ax2)
    deets_x.to_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv')
    deets_y.to_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv')

    return deets_x, deets_y, err_x, err_y

## Multipoint-resampling

In [9]:
def restructure_dfs(experiment_id):
    """
    Restructure details dataframe to have the resampling points as a column, also adds in a column of resampling difference
    :param experiment_id: experiment identifier to use
    :return: None -> saves the resampling dataframe as {experiment_id}_resample_[x,y].csv
    """
    api = wandb.Api()
    runs =  api.runs('cardiors/interpretability',
                     filters={"$and": [{'tags': f'{experiment_id}'}, {'tags': 'resample_multiplicities'}, {'state': 'finished'}]})
    run = runs[0]
    details_df, path = load_details(run)

    # NOISE X INTERVENTION
    dfx = details_df[details_df.modified_attribute == 'noise_x']
    x_colnames = [f'x_intervention{n}' for n in range(100)]
    intervention_xdf = dfx.loc[:, x_colnames]
    intervention_xdf['x_intervention'] = intervention_xdf.values.tolist()
    resampling_xdf = pd.DataFrame(dict(x_orig=dfx.x_orig, y_orig=dfx.y_orig, x_resampling=intervention_xdf.x_intervention, time=dfx.time_orig, event=dfx.event_orig))

    # Resampling difference noise_x
    orig_vals = resampling_xdf.loc[:, 'x_orig'].values
    resamp_vals = resampling_xdf.loc[:, 'x_resampling'].values.tolist()
    resamp_vals_numpy=np.array([np.array(row) for row in resamp_vals])
    orig_vals_array = np.array([[val] * 100 for val in orig_vals])
    resampling_xdf.loc[:, 'resampling_diff'] = np.absolute(orig_vals_array - resamp_vals).tolist()
    resampling_xdf.to_csv(f'/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/resample_multiplicities/{experiment_id}_resampling_x.csv')

    # NOISE Y INTERVENTION
    dfy = details_df[details_df.modified_attribute == 'noise_y']
    y_colnames = [f'y_intervention{n}' for n in range(100)]
    intervention_ydf = dfy.loc[:, y_colnames]
    intervention_ydf['y_intervention'] = intervention_ydf.values.tolist()
    resampling_ydf = pd.DataFrame(dict(x_orig=dfy.x_orig, y_orig=dfy.y_orig, y_resampling=intervention_ydf.y_intervention, time=dfy.time_orig, event=dfy.event_orig))

    # Resampling difference noise_y
    orig_vals = resampling_ydf.loc[:, 'y_orig'].values
    resamp_vals = resampling_ydf.loc[:, 'y_resampling'].values.tolist()
    resamp_vals_numpy=np.array([np.array(row) for row in resamp_vals])
    orig_vals_array = np.array([[val] * 100 for val in orig_vals])
    resampling_ydf.loc[:, 'resampling_diff'] = np.absolute(orig_vals_array - resamp_vals).tolist()
    resampling_ydf.to_csv(f'/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/resample_multiplicities/{experiment_id}_resampling_y.csv')

In [ ]:
experiment_ids = ['p0.00', 'p0.25', 'p0.5', 'p0.75']
for experiment_id in experiment_ids:
    restructure_dfs(experiment_id)

## Change Slope

In [10]:
# CREATE TUPLES FOR MODEL DIFF

def col2numpy(df, colname):
    """
    convert a dataframe column to a numpy array
    :param df: dataframe
    :param colname: column name to convert to numpy array
    :return:
    """
    vals = df.loc[:, colname].values.tolist()
    vals_numpy = np.array([np.array(eval(row)) for row in vals])
    return vals_numpy

def clip_norm2d(array):
    """
    clips and normalizes the array to the range of [-1, 1]
    :param array: array to be normalized
    :return: normalized array
    """
    array_norm = np.zeros((array.shape))
    for i in range(array.shape[1]):
        column = array[:, i]
        outliers = np.percentile(column, [0.5, 99.5])
        clipped_array = np.clip(column, *outliers)
        col_norm = clipped_array / abs(clipped_array).max()
        #TODO: !!!!!FIX THIS!!!!
        array_norm[:, i] = col_norm
    return  array_norm

def clip_norm1d(array):
    """
    clips and normalizes the array to the range of [-1, 1]
    :param array:  array to be normalized
    :return: normalized array
    """
    outliers = np.percentile(array, [0.5, 99.5])
    clipped_array = np.clip(array, *outliers)
    array_norm = clipped_array / abs(clipped_array).max()
    return array_norm

def get_errors(change_slope_array, attr_array):
    """
    calculates squared & absolute error between the attribution value and change_slope
    :param change_slope_array: change slope array (2 dimensional)
    :param attr_array: attribution values array (1 dimensional)
    :return: squared_error, absolute error (both 2 dimensional)
    """
    squared_error = np.zeros((change_slope_array.shape))
    absolute_error = np.zeros((change_slope_array.shape))

    for j in range(change_slope_array.shape[0]):
        for i in range(change_slope_array.shape[1]):
            attr = attr_array[i]
            cs = change_slope_array[j, i]
            squared_error[j, i] = np.square(attr - cs)
            absolute_error[j, i] = np.absolute(attr - cs)

    mae = np.mean(absolute_error, axis=1)
    mse = np.mean(squared_error, axis=1)
    rmse = np.sqrt(np.mean(squared_error, axis=1))
    return squared_error, absolute_error, rmse, mse, mae

In [28]:
def resampling_error(experiment_id, method):
    """
    creates a dataframe with resampling error for each point
    :param experiment_id: experiment identifier
    :param method: feature ablation method to calculate errors on
    :return:
    """

    experiment_id = experiment_id
    api = wandb.Api()
    runs =  api.runs('cardiors/interpretability',
                     filters={"$and": [{'tags': f'{experiment_id}'}, {'tags': 'resample_multiplicities'}, {'state': 'finished'}]})

    # For testing purpose
    runs = [runs[0]]
    for run in runs:
        # load resampling dataframe and model
        xdf = pd.read_csv(f'/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/resample_multiplicities/{experiment_id}_resampling_x.csv', index_col=0)
        ydf = pd.read_csv(f'/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/resample_multiplicities/{experiment_id}_resampling_y.csv', index_col=0)
        model = DeepSurv.load_from_checkpoint(run.config['checkpoint_path'])

        # resampling tensors for xdf
        orig_tensor_x = torch.Tensor(xdf[['x_orig', 'y_orig']].to_numpy(dtype='float64'))
        resamp_vals = col2numpy(xdf, 'x_resampling')
        constant_val = xdf['y_orig'].to_numpy(dtype='float64')
        resamp_tensors_x = []
        for i in range(resamp_vals.shape[1]):
            # modified attribute == noise_x -> resamp_tensor = (resamp, constant)
            resamp_tensor = torch.Tensor(np.concatenate((resamp_vals[:, i].reshape(-1, 1), constant_val.reshape(-1, 1)), axis=1))
            resamp_tensors_x.append(resamp_tensor)

        orig_tensor_y = torch.Tensor(ydf[['x_orig', 'y_orig']].to_numpy(dtype='float64'))
        # resampling tensors for ydf
        resamp_vals = col2numpy(ydf, 'y_resampling')
        constant_val = ydf['x_orig'].to_numpy(dtype='float64')
        resamp_tensors_y = []
        for i in range(resamp_vals.shape[1]):
            # modified attribute == noise_y -> resamp_tensor = (constant, resamp)
            resamp_tensor = torch.Tensor(np.concatenate((constant_val.reshape(-1, 1), resamp_vals[:, i].reshape(-1, 1)), axis=1))
            resamp_tensors_y.append(resamp_tensor)

        # CALCULATE MODEL DIFF + CHANGE SLOPE

        # load attributions
        attr_x, attr_y = load_attributions(run, method=method)
        attr_x = attr_x[:, 0]
        attr_y = attr_y[:, 1]

        orig_output = model(orig_tensor_x)[0].detach().numpy()
        resamp_outputs = None
        for i in range(resamp_vals.shape[1]):
            resamp_output = model(resamp_tensors_x[i])[0].detach().numpy()
            resamp_outputs = resamp_output if resamp_outputs is None else np.concatenate((resamp_outputs, resamp_output), axis=1)

        # calculate change slope
        model_diff = np.subtract(orig_output, resamp_outputs)
        resampling_diff = col2numpy(xdf, 'resampling_diff')
        change_slope = np.true_divide(model_diff, resampling_diff)
        mean_cs = np.mean(change_slope, axis=1)

        xdf.loc[:, 'model_diff'] = model_diff.tolist()
        xdf.loc[:, 'change_slope'] = change_slope.tolist()
        xdf.loc[:, 'mean_cs'] = mean_cs
        xdf.loc[:, 'attribution_x'] = attr_x

        # calculate change slope
        orig_output = model(orig_tensor_y)[0].detach().numpy()
        resamp_outputs = None
        for i in range(resamp_vals.shape[1]):
            resamp_output = model(resamp_tensors_y[i])[0].detach().numpy()
            resamp_outputs = resamp_output if resamp_outputs is None else np.concatenate((resamp_outputs, resamp_output), axis=1)

        model_diff = np.subtract(orig_output, resamp_outputs)
        resampling_diff = col2numpy(ydf, 'resampling_diff')
        change_slope = np.true_divide(model_diff, resampling_diff)
        mean_cs = np.mean(change_slope, axis=1)

        ydf.loc[:, 'model_diff'] = model_diff.tolist()
        ydf.loc[:, 'change_slope'] = change_slope.tolist()
        ydf.loc[:, 'mean_cs'] = mean_cs
        ydf.loc[:, 'attribution_y'] = attr_y

        # Save dataframes
        EVALUATION_DIR = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/results/interpretability/resample_multiplicities/evaluation'
        if not os.path.exists(f'{EVALUATION_DIR}/{experiment_id}'):
            os.mkdir(f'{EVALUATION_DIR}/{experiment_id}')

        if not os.path.exists(f'{EVALUATION_DIR}/{experiment_id}/change_slope'):
            os.mkdir(f'{EVALUATION_DIR}/{experiment_id}/change_slope')

        seed = eval(run.config['_content']['experiment'])['datamodule_kwargs']['seed']
        xdf.to_csv(f'{EVALUATION_DIR}/{experiment_id}/change_slope/x_{method}_{seed}.csv')
        ydf.to_csv(f'{EVALUATION_DIR}/{experiment_id}/change_slope/y_{method}_{seed}.csv')
        return xdf, ydf

In [12]:
methods = ['FeatureAblation',
           'KernelExplainer',
           'DeepExplainer']

In [20]:
experiment_ids = ['p0.75']
xdf, ydf = resampling_error(experiment_id, 'FeatureAblation')

In [21]:
xdf.change_slope[0]

,x_orig,y_orig,x_resampling,time,event,resampling_diff,model_diff,change_slope,mean_cs,attribution_x
0,0.348720,0.389544,"[-0.3359962421121104, 0.0757522714949251, -0.8...",0.791166,1.0,"[0.6847161281312388, 0.2729676145242033, 1.215...","[0.24990850687026978, 0.0982816070318222, 0.47...","[0.36498118943441926, 0.3600485984505236, 0.38...",0.064747,0.115524
1,0.784569,0.446195,"[-0.1530040531730433, 1.087602749330351, 0.819...",0.806355,1.0,"[0.9375727475942667, 0.3030340549091277, 0.034...","[0.3737141489982605, -0.1337873935699463, -0.0...","[0.39859749545534445, -0.44149293256847244, -0...",0.143162,0.271590
2,1.209356,1.314928,"[0.9367276467400668, 0.1407234548299741, 0.212...",0.000000,0.0,"[0.2726279408110882, 1.068632132721181, 0.9970...","[0.0038338899612426758, 0.3645629286766052, 0....","[0.014062718406031937, 0.34114913590355617, 0....",0.163415,0.409826
3,0.233735,-0.274223,"[-1.7405027677917992, -0.5069073641056587, 0.0...",0.878572,1.0,"[1.9742376039544363, 0.7406422002682957, 0.171...","[0.8953638076782227, 0.34671059250831604, 0.10...","[0.45352383415491204, 0.46812157392965326, 0.5...",0.201644,0.079062
4,-0.612208,0.822774,"[1.7694521637651923, -0.4403750472000197, -1.4...",0.920771,1.0,"[2.381660448933591, 0.17183323796837907, 0.837...","[-1.1232879161834717, -0.14560528099536896, 0....","[-0.47164066426279766, -0.8473638902280558, 0....",-0.406306,-0.199007
...,...,...,...,...,...,...,...,...,...,...
5095,-2.713351,-2.496023,"[0.0105750265863676, -0.7937752253749618, -0.4...",1.142518,1.0,"[2.7239258022415243, 1.9195755502801948, 2.251...","[-0.9708305597305298, -0.621554970741272, -0.7...","[-0.3564085919416863, -0.3237981285240612, -0....",-0.351507,-0.945044
5096,0.309199,-0.334600,"[1.4345700560264594, 1.6163265076751128, -0.15...",0.842824,1.0,"[1.1253710448337375, 1.307127496482391, 0.4632...","[-0.4605836868286133, -0.5177865028381348, 0.2...","[-0.40927273626154115, -0.39612547684257987, 0...",0.252234,0.106218
5097,0.748925,0.182532,"[-0.0265191369169419, -0.3005558763207619, 0.5...",0.307402,1.0,"[0.775444065070997, 1.049480804474817, 0.18374...","[0.3347215950489044, 0.46269601583480835, 0.09...","[0.4316515015409377, 0.4408808754404532, 0.514...",0.326889,0.262278
5098,-0.567965,-0.505648,"[-0.5771746694021446, -0.4294065409476704, -1....",0.895207,1.0,"[0.00920977065479156, 0.13855835779968262, 1.1...","[0.03597268462181091, -0.012654811143875122, 0...","[3.905926213601794, -0.09133199429348396, 0.47...",-0.016208,-0.191115


In [18]:
xdf.model_diff

0        [0.26356083154678345]
1         [0.2820919156074524]
2        [0.14521509408950806]
3       [-0.42148637771606445]
4       [-0.24262291193008423]
                 ...          
5095     [-1.5800204277038574]
5096     [-0.1761873960494995]
5097      [1.0370593070983887]
5098       [0.609470546245575]
5099    [-0.06177254021167755]
Name: model_diff, Length: 5100, dtype: object

In [ ]:
method = ['FeatureAblation']
experiment_id = ['p0.70']
xdf, ydf = resampling_error(experiment_id, method)

In [ ]:
experiment_ids = ['p0.75']
for method in methods:
    for experiment_id in experiment_ids:
        resampling_error(experiment_id, method)


## Experiments

In [ ]:
deets_x, deets_y, err_x, err_y = get_mean_error('p0.00', method='FeatureAblation')

In [ ]:
pd.DataFrame(err_x)

In [ ]:
deets_x

## p = 0.00

In [ ]:
methods = ['FeatureAblation',
           'FeaturePermutation',
           'KernelExplainer',
           'DeepExplainer',
           'ShapleyValueSampling',
           'IntegratedGradients',
           'InputXGradient',
           'Lime']

## p = -0.75

In [ ]:
for method in methods:
    _ = get_mean_error(experiment_id='p-0.75', method=method)

In [ ]:
import seaborn as sns

In [ ]:
method = 'KernelExplainer'

def plot_distributions(method):
    experiment_id = 'p0.00'
    xdf_00 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_00 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_00, ydf_00]:
        df.loc[:, 'correlation_coefficient'] = 0.00

    experiment_id = 'p-0.75'
    xdf_75 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_75 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_75, ydf_75]:
        df.loc[:, 'correlation_coefficient'] = 0.75

    experiment_id = 'p0.30'
    xdf_30 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_30 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_30, ydf_30]:
        df.loc[:, 'correlation_coefficient'] = 0.30

    experiment_id = 'p0.50'
    xdf_50 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_50 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_50, ydf_50]:
        df.loc[:, 'correlation_coefficient'] = 0.50

    experiment_id = 'p0.20'
    xdf_20 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_20 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_20, ydf_20]:
        df.loc[:, 'correlation_coefficient'] = 0.20

    full_xdf = xdf_75.append([xdf_00, xdf_20, xdf_30, xdf_50], ignore_index=True)
    full_ydf = ydf_75.append([xdf_00, ydf_20, ydf_30, ydf_50], ignore_index=True)
    full_df = full_xdf.append(full_ydf, ignore_index=True)
    sns.set(rc={'figure.figsize':(20,10)})
    sns.displot(data=full_df,
                x='mean_error',
                hue="correlation_coefficient",
                col='modified_attribute',
                fill=True,
                kind='kde',
                palette='crest',
                alpha=0.1,
                linewidth=2.0,
                legend=False)
    plt.suptitle(method, y=0.99, x=0.54)
plot_distributions(method)

In [ ]:
for method in methods:
    plot_distributions(method)

In [ ]:
full_df[full_df.modified_attribute=='noise_x' ]

In [ ]:
full_df[full_df.attribution_method=='DeepExplainer'].nlargest(100, ['mean_error'])['x_orig'].index

In [ ]:
experiment_id = 'p0.00'
method = 'DeepExplainer'
xdf_de = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
ydf_de = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
for df in [xdf_de, ydf_de]:
    df.loc[:, 'attribution_method'] = method

In [ ]:
noise_ydf = full_df[full_df.modified_attribute == 'noise_y']

In [ ]:
noise_ydf[noise_ydf.attribution_method == 'DeepExplainer']

In [ ]:
full_df = compare_attributions('p-0.75', dimension='y')

In [ ]:
method = 'KernelExplainer'

def compare_attributions(experiment_id:str, dimension:str = 'both') -> pd.DataFrame:
    method = 'FeatureAblation'
    xdf_fa = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_fa = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_fa, ydf_fa]:
        df.loc[:, 'attribution_method'] = method

    method = 'FeaturePermutation'
    xdf_fp = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_fp = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_fp, ydf_fp]:
        df.loc[:, 'attribution_method'] = method

    method = 'KernelExplainer'
    xdf_ke = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_ke = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_ke, ydf_ke]:
        df.loc[:, 'attribution_method'] = method

    method = 'DeepExplainer'
    xdf_de = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_de = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_de, ydf_de]:
        df.loc[:, 'attribution_method'] = method

    method = 'ShapleyValueSampling'
    xdf_svs = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_svs = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_svs, ydf_svs]:
        df.loc[:, 'attribution_method'] = method

    method = 'IntegratedGradients'
    xdf_ig = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_ig = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_ig, ydf_ig]:
        df.loc[:, 'attribution_method'] = method

    method = 'InputXGradient'
    xdf_ixg = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_ixg = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_ixg, ydf_ixg]:
        df.loc[:, 'attribution_method'] = method

    method = 'Lime'
    xdf_lime = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_lime = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_lime, ydf_lime]:
        df.loc[:, 'attribution_method'] = method

    full_xdf = xdf_de.append([xdf_lime, xdf_ixg, xdf_ig, xdf_fa, xdf_fp, xdf_ke, xdf_svs], ignore_index=True)
    full_ydf = ydf_de.append([ydf_lime, ydf_ixg, ydf_ig, ydf_fa, ydf_fp, ydf_ke, ydf_svs], ignore_index=True)
    full_df = full_xdf.append(full_ydf, ignore_index=True)

    full_df = full_xdf if dimension=='x' else full_ydf if dimension=='y' else full_xdf.append(full_ydf, ignore_index=True)
    sns.set(rc={'figure.figsize':(20,10)})
    sns.boxplot(data=full_df,
                x='attribution_method',
                y="mean_error",
                hue='modified_attribute',
                palette='Spectral_r')
    plt.suptitle(experiment_id, y=0.99, x=0.54)
    return full_df


def plot_distributions(method):
    experiment_id = 'p0.00'
    xdf_00 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_00 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_00, ydf_00]:
        df.loc[:, 'correlation_coefficient'] = 0.00

    experiment_id = 'p-0.75'
    xdf_75 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_75 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_75, ydf_75]:
        df.loc[:, 'correlation_coefficient'] = 0.75

    experiment_id = 'p0.30'
    xdf_30 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_30 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_30, ydf_30]:
        df.loc[:, 'correlation_coefficient'] = 0.30

    experiment_id = 'p0.50'
    xdf_50 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_50 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_50, ydf_50]:
        df.loc[:, 'correlation_coefficient'] = 0.50

    experiment_id = 'p0.20'
    xdf_20 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_x_{method}.csv', index_col=0)
    ydf_20 = pd.read_csv(f'{RESULTS_DIR}/{experiment_id}/change_slope/MeanError_y_{method}.csv', index_col=0)
    for df in [xdf_20, ydf_20]:
        df.loc[:, 'correlation_coefficient'] = 0.20

    full_xdf = xdf_75.append([xdf_00, xdf_20, xdf_30, xdf_50], ignore_index=True)
    full_ydf = ydf_75.append([xdf_00, ydf_20, ydf_30, ydf_50], ignore_index=True)
    full_df = full_xdf.append(full_ydf, ignore_index=True)
    sns.set(rc={'figure.figsize':(20,10)})
    sns.displot(data=full_df,
                x='mean_error',
                hue="correlation_coefficient",
                col='modified_attribute',
                fill=True,
                kind='kde',
                palette='crest',
                alpha=0.1,
                linewidth=2.0,
                legend=False)
    plt.suptitle(method, y=0.99, x=0.54)
plot_distributions(method) 1